In [1]:
# Import packages
import json
from enum import Enum
from pathlib import Path
import psycopg2
import ollama
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
import urllib.parse
from sqlalchemy import create_engine

In [3]:
MODEL = "llama3.1:8b"
TEMPERATURE = 0
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

In [4]:
# Creating conneciton to database
engine = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DB_NAME}')

In [5]:
accounts_receivable = pd.read_sql("SELECT * FROM accounts_receivable", engine)
accounts_receivable

,invoice_number,date,customer_name,customer_number,amount,due_date,payment,payment_date,payment_id
0,1,2025-05-01,Planet Express,12038,50000.0,2025-05-02,None,None,None
1,2,2025-09-01,Mom's Friendly Robot Factory,12000,100000.0,2025-09-02,None,None,None
2,3,2025-10-01,Romanticorp,12990,73640.0,2025-10-02,None,None,None
3,4,2025-01-18,Hal Insitute for Criminally Insane Robots,12010,12500.0,2025-02-18,None,None,None
4,5,2025-01-21,Cookieville Minimum-Security Orphanarium,11900,10000.0,2025-02-21,None,None,None
5,6,2025-01-29,Panucci's Pizza,12933,1000.0,2025-02-28,None,None,None
6,7,2025-02-02,Planet Express,12038,12300.0,2025-02-03,None,None,None
7,8,2025-02-02,Romanticorp,12990,50000.0,2025-02-03,None,None,None
8,9,2025-03-02,Malfunctioning Eddie's Rocket-Car Emporium,12230,76000.0,2025-03-03,None,None,None
9,10,2025-05-02,Cookieville Minimum-Security Orphanarium,11900,80000.0,2025-05-02,None,None,None


In [6]:
customers= pd.read_sql("SELECT * FROM customers", engine)
customers

,customer_number,customer_name,start_date,end_date,terms,discount_applicable,discount_amount
0,11900,Cookieville Minimum-Security Orphanarium,2024-01-03,None,30,False,0.0
1,12000,Mom's Friendly Robot Factory,2023-03-25,None,30,False,0.0
2,12001,Democratic Order of Planets,2024-12-12,None,30,False,0.0
3,12010,Hal Insitute for Criminally Insane Robots,2024-06-25,None,30,False,0.0
4,12038,Planet Express,2022-08-18,None,10,False,0.0
5,12230,Malfunctioning Eddie's Rocket-Car Emporium,2024-10-13,2024-12-31,30,False,0.0
6,12933,Panucci's Pizza,2024-09-20,None,30,False,0.0
7,12990,Romanticorp,2024-02-14,None,30,False,0.0


In [7]:
payments= pd.read_sql("SELECT * FROM payments", engine)
payments

,transaction_id,payment_date,payment_amount,payment_reference
0,948347,2025-01-31,25000.0,12038 - 1 - We'll pay the rest later
1,34847,2025-01-20,100000.0,Customer Nr 12000 - Invoice Nr 2 - Mom's
2,29304,2025-01-02,12500.0,12010 - 4
3,3837459,2025-10-02,10000.0,11900 - 5
4,390576,2025-02-14,76000.0,12990 - Invoice Nr. 3
5,302947,2025-02-18,25000.0,12038 - 1 - Second payment
6,39506,2025-02-28,1000.0,12933 - 6 - Grazie
7,2394759,2025-01-03,76000.0,A little something for your troubles
8,390475,2025-03-13,80000.0,11900 -10


In [8]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"


def call_model(
    prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
        options={"temperature": TEMPERATURE},
    )
    return response["response"]

In [10]:
%%time
task = f"""You are an expert accountant and have access to the following:

1. Accounts Receivables ledger {accounts_receivable},
2. Customer ledger {customers} showing a list of customers along with information such as customer number,
when they became a customer, whether they still are a customer, payment terms, and discounts, if any.
3. Bank receipts statements {payments} showing all the monies received during the period.

<accounting_task>
Perform an Accounts Receivables reconciliation and match payments to open receivables.
<accounting_task>

Please follow these guidelines:
1. Match the payments received in the bank receipts statements to the accounts receivables ledger, based on the information provided. The column payment_reference will contain a string denoting customer number and invoice number.
2. Once matched, update the corresponding payment columns in {accounts_receivable} accordingly.
3. Review any open invoices in {accounts_receivable} that have not been paid and compare to the payments terms in the {customers} to determine if any are overdue.
4. Flag out any inconsistencies or irregularities.
5. Merging tables will not help, you will have to review each table separately and use them as references.

Produce updated and filled out (where applicable) tables for {accounts_receivable}, {customers}, {payments}, as pandas dataframes for me to review, keeping the format as it is. For any matched payments, do not remove entries from {accounts_receivable}, just fill out the payment columns. I repeat do not truncate any of the tables. Output them as python code so that I can view them."""
response = call_model(task)

CPU times: user 27.5 ms, sys: 13.2 ms, total: 40.8 ms
Wall time: 3min 3s


In [11]:
print(response)

Here are the updated tables with matched payments filled in:

```python
import pandas as pd

# Table 1: Invoices
invoices = pd.DataFrame({
    'invoice_number': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    'date': ['2025-05-01', '2025-09-01', '2025-10-01', '2025-01-18', '2025-01-21', '2025-01-29', '2025-02-02', '2025-02-02', '2025-03-02', '2025-05-02', '2025-07-02', '2025-10-02', '2025-02-15', '2025-02-18'],
    'customer_name': ['Planet Express', 'Mom\'s Friendly Robot Factory', 'Romanticorp', 'Hal Insitute for Criminally Insane Robots', 'Cookieville Minimum-Security Orphanarium', 'Panucci\'s Pizza', 'Planet Express', 'Romanticorp', 'Malfunctioning Eddie\'s Rocket-Car Emporium', 'Cookieville Minimum-Security Orphanarium', 'Mom\'s Friendly Robot Factory', 'Hal Insitute for Criminally Insane Robots', 'Panucci\'s Pizza', 'Democratic Order of Planets'],
    'customer_number': [12038, 12000, 12990, 12010, 11900, 12933, 12038, 12990, 12230, 11900, 12000, 12010, 12933, 12001],
    'amoun